In [ ]:
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.linear_model import LinearRegression, Ridge, Lasso, SGDClassifier, LogisticRegression
from sklearn.linear_model import RidgeClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score, f1_score, recall_score, roc_auc_score, precision_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import cross_validate
import xgboost as xgb
import lightgbm as lgb

import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning, module="my_module")
import warnings
warnings.simplefilter('ignore')


In [ ]:
import matplotlib.pylab as plt
from scipy import interp
from sklearn.linear_model import LogisticRegression, Lasso, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import KFold
import matplotlib.patches as patches
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.preprocessing import LabelBinarizer

datasets = [(X_0_5s_c, y_0_5s_c), (X_1s_c, y_1s_c), (X_2s_c, y_2s_c), (X_3s_c, y_3s_c), (X_4s_c, y_4s_c), (X_5s_c, y_5s_c)]
labels = [0.5, 1, 2, 3, 4, 5]
random_state = np.random.RandomState(0)
np.random.seed(0)

fig1, ax = plt.subplots(ncols=3, figsize=[32, 24], nrows=2, sharex=True, sharey=True)

class_combinations = [(0, [1, 2]), (1, [0, 2]), (2, [0, 1])]
class_labels = ['Baseline vs (CTA, CTA+RTA)', 'CTA vs (Baseline, CTA+RTA)', 'CTA+RTA vs (Baseline, CTA)']

for i, (x, y) in enumerate(datasets):
    print('Processing', i)
    classifiers = [
        ('Logistic Regression', LogisticRegression(random_state=random_state)),
        ('MLP Classifier', MLPClassifier(random_state=random_state)),
        ('Linear Discriminant Analysis', LinearDiscriminantAnalysis()),
        ('KNN', KNeighborsClassifier()),
        ('Decision Tree', DecisionTreeClassifier(random_state=random_state)),
        ('Random Forest', RandomForestClassifier(random_state=random_state)),
        ('AdaBoost', AdaBoostClassifier(random_state=random_state)),
        ('XGBoost', xgb.XGBClassifier(random_state=random_state)),
        ('LightGBM', lgb.LGBMClassifier(random_state=random_state))
    ]

    mean_fpr = np.linspace(0, 1, 100)

    mean_tprs_micro = []
    mean_aucs_micro = []

    for name, clf in classifiers:
        cv = KFold(n_splits=5, shuffle=True)
        tprs_micro = []
        aucs_micro = []

        for idx, (class_idx, rest_classes) in enumerate(class_combinations):
            for train, test in cv.split(x, y):
                clf.fit(x.iloc[train], y.iloc[train])
                prediction = clf.predict_proba(x.iloc[test])

                lb = LabelBinarizer()
                lb.fit(y)
                y_test_bin = lb.transform(y.iloc[test])
                y_test_bin_one_vs_rest = np.hstack((y_test_bin[:, class_idx].reshape(-1, 1), y_test_bin[:, rest_classes]))

                fpr_micro, tpr_micro, _ = roc_curve(y_test_bin.ravel(), prediction.ravel())
                tprs_micro.append(interp(mean_fpr, fpr_micro, tpr_micro))
                roc_auc_micro = auc(fpr_micro, tpr_micro)
                aucs_micro.append(roc_auc_micro)

        mean_tpr_micro = np.mean(tprs_micro, axis=0)
        mean_auc_micro = np.mean(aucs_micro)
        mean_tprs_micro.append(mean_tpr_micro)
        mean_aucs_micro.append(mean_auc_micro)

        ax.flat[i].plot(mean_fpr, mean_tpr_micro, label='%s(AUC = %0.3f)' % (name, mean_auc_micro), lw=2)

    ax.flat[i].plot([0, 1], [0, 1], linestyle='--', lw=2, color='black')
    ax.flat[i].set_xlabel('False Positive Rate', fontweight='bold', fontsize='13')
    ax.flat[i].set_ylabel('True Positive Rate', fontweight='bold', fontsize='13')
    ax.flat[i].set_title('ROC curve in ' + str(labels[i]) + 's window', fontweight='bold', fontsize='13')
    ax.flat[i].legend(loc='lower right', prop={'weight': 'bold', 'size': '13'})
plt.savefig('roc_curves.png', dpi=300, bbox_inches='tight')

plt.show()


In [ ]:
import matplotlib.pylab as plt
from scipy import interp
from sklearn.linear_model import LogisticRegression, Lasso, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import KFold
import matplotlib.patches as patches
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.preprocessing import LabelBinarizer

datasets = [(X_0_5s_d, y_0_5s_d), (X_1s_d, y_1s_d), (X_2s_d, y_2s_d), (X_3s_d, y_3s_d), (X_4s_d, y_4s_d), (X_5s_d, y_5s_d)]
labels = [0.5, 1, 2, 3, 4, 5]
random_state = np.random.RandomState(0)
np.random.seed(0)

fig1, ax = plt.subplots(ncols=3, figsize=[32, 24], nrows=2, sharex=True, sharey=True)

class_combinations = [(0, [1, 2]), (1, [0, 2]), (2, [0, 1])]
class_labels = ['Baseline vs (Attention, Distraction)', 'Distraction vs (Baseline, Attention)', 'Attention vs (Baseline, Distraction)']

for i, (x, y) in enumerate(datasets):
    print('Processing', i)
    classifiers = [
        ('Logistic Regression', LogisticRegression(random_state=random_state)),
        ('MLP Classifier', MLPClassifier(random_state=random_state)),
        ('Linear Discriminant Analysis', LinearDiscriminantAnalysis()),
        ('KNN', KNeighborsClassifier()),
        ('Decision Tree', DecisionTreeClassifier(random_state=random_state)),
        ('Random Forest', RandomForestClassifier(random_state=random_state)),
        ('AdaBoost', AdaBoostClassifier(random_state=random_state)),
        ('XGBoost', xgb.XGBClassifier(random_state=random_state)),
        ('LightGBM', lgb.LGBMClassifier(random_state=random_state))
    ]

    mean_fpr = np.linspace(0, 1, 100)

    mean_tprs_micro = []
    mean_aucs_micro = []

    for name, clf in classifiers:
        cv = KFold(n_splits=5, shuffle=True)
        tprs_micro = []
        aucs_micro = []

        for idx, (class_idx, rest_classes) in enumerate(class_combinations):
            for train, test in cv.split(x, y):
                clf.fit(x.iloc[train], y.iloc[train])
                prediction = clf.predict_proba(x.iloc[test])

                lb = LabelBinarizer()
                lb.fit(y)
                y_test_bin = lb.transform(y.iloc[test])
                y_test_bin_one_vs_rest = np.hstack((y_test_bin[:, class_idx].reshape(-1, 1), y_test_bin[:, rest_classes]))

                fpr_micro, tpr_micro, _ = roc_curve(y_test_bin.ravel(), prediction.ravel())
                tprs_micro.append(interp(mean_fpr, fpr_micro, tpr_micro))
                roc_auc_micro = auc(fpr_micro, tpr_micro)
                aucs_micro.append(roc_auc_micro)

        mean_tpr_micro = np.mean(tprs_micro, axis=0)
        mean_auc_micro = np.mean(aucs_micro)
        mean_tprs_micro.append(mean_tpr_micro)
        mean_aucs_micro.append(mean_auc_micro)

        ax.flat[i].plot(mean_fpr, mean_tpr_micro, label='%s(AUC = %0.3f)' % (name, mean_auc_micro), lw=2)

    ax.flat[i].plot([0, 1], [0, 1], linestyle='--', lw=2, color='black')
    ax.flat[i].set_xlabel('False Positive Rate', fontweight='bold', fontsize='13')
    ax.flat[i].set_ylabel('True Positive Rate', fontweight='bold', fontsize='13')
    ax.flat[i].set_title('ROC curve in ' + str(labels[i]) + 's window', fontweight='bold', fontsize='13')
    ax.flat[i].legend(loc='lower right', prop={'weight': 'bold', 'size': '13'})
plt.savefig('roc_curves.png', dpi=300, bbox_inches='tight')

plt.show()


In [ ]:

from sklearn.multiclass import OneVsRestClassifier

def benchmark_models_3(X, y):
    models = {
        'Logistic Regression': LogisticRegression(multi_class='auto'),
        'Ridge Classifier': RidgeClassifier(),
        'SGD Classifier': SGDClassifier(),
        'Support Vector Classifier': SVC(),
        'Decision Tree': DecisionTreeClassifier(),
        'Random Forest': RandomForestClassifier(),
        'AdaBoost': AdaBoostClassifier(),
        'MLP Classifier': MLPClassifier(),
        'Linear Discriminant Analysis': LinearDiscriminantAnalysis(),
        'KNN': KNeighborsClassifier(),
        'XGBoost': xgb.XGBClassifier(objective='multi:softmax'),
        'LightGBM': lgb.LGBMClassifier()
    }

    cv = KFold(n_splits=5, shuffle=True, random_state=1)

    scoring = {'accuracy': 'accuracy',
               'f1_score_micro': 'f1_micro',
               'f1_score_macro': 'f1_macro',
               'recall_micro': 'recall_micro',
               'recall_macro': 'recall_macro',
               'precision_micro': 'precision_micro',
               'precision_macro': 'precision_macro'}

    results = pd.DataFrame(columns=['Accuracy', 'F1 Score ', 'Recall ', 'Precision'])

    for name, model in models.items():
        if isinstance(model, (LogisticRegression, RidgeClassifier, SGDClassifier, SVC)):
            # Use the "ovr" strategy for binary relevance
            model = OneVsRestClassifier(model)
        elif isinstance(model, (lgb.LGBMClassifier)):
            # Set the "objective" parameter to "multiclass" for multi-class classification
            model = model.set_params(objective='multiclass')

        scores = cross_validate(model, X, y, cv=cv, scoring=scoring)
        results.loc[name] = [
            scores['test_accuracy'].mean(),
            #scores['test_f1_score_micro'].mean(),
            scores['test_f1_score_macro'].mean(),
            #scores['test_recall_micro'].mean(),
            scores['test_recall_macro'].mean(),
            #scores['test_precision_micro'].mean(),
            scores['test_precision_macro'].mean()
        ]

    return results


In [ ]:
benchmark_models_3(X_1s_d, y_1s_d)

In [ ]:
benchmark_models_3(X_1s_c, y_1s_c)

In [ ]:
items = (['Temporal & Movement'] * 5) + (['Gaze & Pupil'] * 45) + (['Head Movement & Orientation'] * 17)

####Distraction

In [ ]:
import eli5
from eli5.sklearn import PermutationImportance
from sklearn.model_selection import KFold
import lightgbm as lgb
import numpy as np
import pandas as pd

RANDOM_STATE = 1
kf = KFold(n_splits=5)
clf = lgb.LGBMClassifier()
X, y = X_1s_d, y_1s_d
perm_importances = []

for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    clf.fit(X_train, y_train, eval_set=[(X_test, y_test)])
    perm = PermutationImportance(clf, random_state=RANDOM_STATE).fit(X_test, y_test)
    perm_importances.append(perm)

# Calculate the importance scores for each fold
importance_scores = np.array([perm.feature_importances_ for perm in perm_importances])

# Calculate summary statistics for each feature
min_scores = np.min(importance_scores, axis=0)
q1_scores = np.percentile(importance_scores, 25, axis=0)
mean_scores = np.mean(importance_scores, axis=0)
median_scores = np.median(importance_scores, axis=0)
q3_scores = np.percentile(importance_scores, 75, axis=0)
max_scores = np.max(importance_scores, axis=0)

# Create a DataFrame with feature names as index
summary_df = pd.DataFrame({
    'Min': min_scores,
    'Q1': q1_scores,
    'Mean': mean_scores,
    'Median': median_scores,
    'Q3': q3_scores,
    'Max': max_scores
}, index=X.columns)

# Print the summary DataFrame
print(summary_df)


In [ ]:
summary_df['group'] = items
summary_df = summary_df.reset_index()
summary_df = summary_df.rename(columns={'index': 'label'})

In [ ]:
sorted_grouped_df = summary_df.groupby('group', group_keys=False).apply(lambda group: group.sort_values(by='Median', ascending=False))

In [ ]:
filtered_df = sorted_grouped_df.groupby('group').head(5)
filtered_df = filtered_df.reset_index(drop = True)
filtered_df_2 = filtered_df
filtered_df_2['Median'] = filtered_df_2['Median'] * 100
filtered_df_2['Q1'] = filtered_df_2['Q1'] * 100
filtered_df_2['Q3'] = filtered_df_2['Q3'] * 100

In [ ]:
pip install forestplot
import forestplot as fp

In [ ]:
fp.forestplot(filtered_df_2,  # the dataframe with results data
              estimate='Median',  # col containing estimated effect size
              ll='Q1', hl='Q3',  # columns containing conf. int. lower and higher limits
              varlabel='label',  # column containing variable label
              capitalize="capitalize",  # Capitalize labels
              groupvar="group",  # Add variable groupings
              #decimal_precision=4,
              group_order=['Temporal & Movement', 'Gaze & Pupil', 'Head Movement & Orientation'],
              #ylabel="Confidence interval",  # y-label title
              xlabel="Decrease in Accuracy Score % ",  # x-label title
              **{#"marker": "D",  # set maker symbol as diamond
                 #"markersize": 35,  # adjust marker size
                 "xlinestyle": (0, (10, 5)),  # long dash for x-reference line
                 "xlinecolor": "#808080",  # gray color for x-reference line
                 "xtick_size": 12,  # adjust x-ticker fontsize
                }
              )

####CTA/RTA

In [ ]:
import eli5
from eli5.sklearn import PermutationImportance
from sklearn.model_selection import KFold
import lightgbm as lgb
import numpy as np
import pandas as pd

RANDOM_STATE = 1
kf = KFold(n_splits=5)
clf = lgb.LGBMClassifier()
X, y = X_1s_c, y_1s_c
perm_importances = []

for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    clf.fit(X_train, y_train, eval_set=[(X_test, y_test)])
    perm = PermutationImportance(clf, random_state=RANDOM_STATE).fit(X_test, y_test)
    perm_importances.append(perm)

importance_scores = np.array([perm.feature_importances_ for perm in perm_importances])

min_scores = np.min(importance_scores, axis=0)
q1_scores = np.percentile(importance_scores, 25, axis=0)
mean_scores = np.mean(importance_scores, axis=0)
median_scores = np.median(importance_scores, axis=0)
q3_scores = np.percentile(importance_scores, 75, axis=0)
max_scores = np.max(importance_scores, axis=0)

summary_df = pd.DataFrame({
    'Min': min_scores,
    'Q1': q1_scores,
    'Mean': mean_scores,
    'Median': median_scores,
    'Q3': q3_scores,
    'Max': max_scores
}, index=X.columns)


In [ ]:
summary_df['group'] = items
summary_df = summary_df.reset_index()
summary_df = summary_df.rename(columns={'index': 'label'})


In [ ]:
sorted_grouped_df = summary_df.groupby('group', group_keys=False).apply(lambda group: group.sort_values(by='Median', ascending=False))

In [ ]:
filtered_df = sorted_grouped_df.groupby('group').head(5)
filtered_df = filtered_df.reset_index(drop = True)
filtered_df_2 = filtered_df
filtered_df_2['Median'] = filtered_df_2['Median'] * 100
filtered_df_2['Q1'] = filtered_df_2['Q1'] * 100
filtered_df_2['Q3'] = filtered_df_2['Q3'] * 100

In [ ]:
pip install forestplot
import forestplot as fp

In [ ]:
fp.forestplot(filtered_df_2,  # the dataframe with results data
              estimate='Median',  # col containing estimated effect size
              ll='Q1', hl='Q3',  # columns containing conf. int. lower and higher limits
              varlabel='label',  # column containing variable label
              capitalize="capitalize",  # Capitalize labels
              groupvar="group",  # Add variable groupings
              #decimal_precision=4,
              group_order=['Temporal & Movement', 'Gaze & Pupil', 'Head Movement & Orientation'],
              #ylabel="Confidence interval",  # y-label title
              xlabel="Decrease in Accuracy Score % ",  # x-label title
              **{#"marker": "D",  # set maker symbol as diamond
                 #"markersize": 35,  # adjust marker size
                 "xlinestyle": (0, (10, 5)),  # long dash for x-reference line
                 "xlinecolor": "#808080",  # gray color for x-reference line
                 "xtick_size": 12,  # adjust x-ticker fontsize
                }
              )